In [ ]:
import numpy as np # linear algebra
np.random.seed(666)
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import datetime
import time
from sklearn.model_selection import train_test_split
from sklearn.metrics import log_loss
from subprocess import check_output
from sklearn.preprocessing import normalize
print(check_output(["ls", "../input"]).decode("utf8"))
start_time = datetime.datetime.now()

In [ ]:
#Load data
train = pd.read_json("../input/train.json")
test = pd.read_json("../input/test.json")
train.inc_angle = train.inc_angle.replace('na', 0)
train.inc_angle = train.inc_angle.astype(float).fillna(0.0)
print("done!")

In [ ]:
# Train data
x_band1 = np.array([np.array(band).astype(np.float32).reshape(75, 75) for band in train["band_1"]])
x_band2 = np.array([np.array(band).astype(np.float32).reshape(75, 75) for band in train["band_2"]])
X_train = np.concatenate([x_band1[:, :, :, np.newaxis]
                          , x_band2[:, :, :, np.newaxis]
                         , ((x_band1+x_band1)/2)[:, :, :, np.newaxis]], axis=-1)
X_angle_train = np.array(train.inc_angle)
y_train = np.array(train["is_iceberg"])

# Test data
x_band1 = np.array([np.array(band).astype(np.float32).reshape(75, 75) for band in test["band_1"]])
x_band2 = np.array([np.array(band).astype(np.float32).reshape(75, 75) for band in test["band_2"]])
X_test = np.concatenate([x_band1[:, :, :, np.newaxis]
                          , x_band2[:, :, :, np.newaxis]
                         , ((x_band1+x_band1)/2)[:, :, :, np.newaxis]], axis=-1)
X_angle_test = np.array(test.inc_angle)

#Reshape the datasets

X_train = X_train.reshape(1604,16875).astype('float32')
X_test = X_test.reshape(8424,16875).astype('float32')

#Nomalize the the data sets:
X_train = normalize(X_train, axis=0, norm='max')
X_angle_train = normalize(X_angle_train, norm='max')
X_test = normalize(X_test, axis=0, norm='max')
X_angle_test = normalize(X_angle_test, norm='max')

#Reshape to original dimensions, to fit into the model

X_train = X_train.reshape(1604,75,75,3).astype('float32')
X_angle_train = X_angle_train.reshape(1604).astype('float32')
X_test = X_test.reshape(8424,75,75,3).astype('float32')
X_angle_test = X_angle_test.reshape(8424).astype('float32')

In [ ]:
X_train, X_valid, X_angle_train, X_angle_valid, y_train, y_valid = train_test_split(X_train
                    , X_angle_train, y_train, random_state=123, train_size=0.8)

In [ ]:
from matplotlib import pyplot
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Dense, Dropout, Input, Flatten
from keras.layers import GlobalMaxPooling2D
from keras.layers.normalization import BatchNormalization
from keras.layers.merge import Concatenate
from keras.models import Model
from keras.optimizers import Adam
from keras.callbacks import ModelCheckpoint, Callback, EarlyStopping

def get_callbacks(filepath, patience=2):
    es = EarlyStopping('val_loss', patience=patience, mode="min")
    msave = ModelCheckpoint(filepath, save_best_only=True)
    return [es, msave]
    
def get_model():
    bn_model = 0
    p_activation = "elu"
    input_1 = Input(shape=(75, 75, 3), name="X_1")
    input_2 = Input(shape=[1], name="angle")
    
    img_1 = Conv2D(16, kernel_size = (3,3), activation=p_activation) ((BatchNormalization(momentum=bn_model))(input_1))
    img_1 = Conv2D(16, kernel_size = (3,3), activation=p_activation) (img_1)
    img_1 = MaxPooling2D((2,2)) (img_1)
    img_1 = Dropout(0.2)(img_1)
    
    img_1 = Conv2D(32, kernel_size = (3,3), activation=p_activation) (img_1)
    img_1 = Conv2D(32, kernel_size = (3,3), activation=p_activation) (img_1)
    img_1 = MaxPooling2D((2,2)) (img_1)
    img_1 = Dropout(0.2)(img_1)
    
    img_1 = Conv2D(64, kernel_size = (3,3), activation=p_activation) (img_1)
    img_1 = Conv2D(64, kernel_size = (3,3), activation=p_activation) (img_1)
    img_1 = MaxPooling2D((2,2)) (img_1)
    img_1 = Dropout(0.2)(img_1)
    
    img_1 = Conv2D(128, kernel_size = (3,3), activation=p_activation) (img_1)
    img_1 = MaxPooling2D((2,2)) (img_1)
    img_1 = Dropout(0.2)(img_1)
    img_1 = GlobalMaxPooling2D() (img_1)
    
    
    img_2 = Conv2D(128, kernel_size = (3,3), activation=p_activation) ((BatchNormalization(momentum=bn_model))(input_1))
    img_2 = MaxPooling2D((2,2)) (img_2)
    img_2 = Dropout(0.2)(img_2)
    img_2 = GlobalMaxPooling2D() (img_2)
    
    img_concat =  (Concatenate()([img_1, img_2, BatchNormalization(momentum=bn_model)(input_2)]))
    
    dense_ayer = Dropout(0.5) (BatchNormalization(momentum=bn_model) ( Dense(256, activation=p_activation)(img_concat) ))
    dense_ayer = Dropout(0.5) (BatchNormalization(momentum=bn_model) ( Dense(64, activation=p_activation)(dense_ayer) ))
    op_activation = "sigmoid"
    output = Dense(1, activation=op_activation)(dense_ayer)
    #output = Dense(1, activation="softmax")(dense_ayer)
    
    model = Model([input_1,input_2],  output)
    optimizer = Adam(lr=0.0001, beta_1=0.9, beta_2=0.999, epsilon=1e-08, decay=0.0)
    model.compile(loss="binary_crossentropy", optimizer=optimizer, metrics=["accuracy"])
    
    model_parameters = {"h_layer_activation":p_activation,"op_layer_activation":op_activation,"optimizer":optimizer
                       }
    return model,model_parameters

model,model_parameters = get_model()
model.summary()


In [ ]:
file_path = ".model_weights.hdf5"
callbacks = get_callbacks(filepath=file_path, patience=5)

model,model_parameters = get_model()
batch_size = 32
model.fit([X_train, X_angle_train], y_train, epochs=25
          , validation_data=([X_valid, X_angle_valid], y_valid)
         , batch_size=batch_size
         , callbacks=callbacks)

In [ ]:
model.load_weights(filepath=file_path)

print("Train evaluate:")
#print(model.evaluate([X_train, X_angle_train], y_train, verbose=1, batch_size=200))
model_eval = model.evaluate([X_train, X_angle_train], y_train, verbose=1, batch_size=200)
print("####################")
print("Watch list evaluate:")
#print(model.evaluate([X_valid, X_angle_valid], y_valid, verbose=1, batch_size=200))
watch_list_eval = model.evaluate([X_valid, X_angle_valid], y_valid, verbose=1, batch_size=200)

In [ ]:
prediction = model.predict([X_test, X_angle_test], verbose=1, batch_size=200)

In [ ]:
submission = pd.DataFrame({'id': test["id"], 'is_iceberg': prediction.reshape((prediction.shape[0]))})
submission.head(10)

In [ ]:
#Add timestamp to file
now = datetime.datetime.now()
filename= str(now.minute)+str(now.hour)+str(now.day)+str(now.month)+str(now.year)+"_submission.csv"
submission.to_csv("../output/"+filename, index=False)

#### Writing information to a meta-file

In [ ]:
#Create a dictionary to write to a file 
run_dict = {"Date":time.strftime("%c"),"model_parameters":model_parameters,"batch_size":batch_size,
           "training_evaluation":model_eval,"watchlist_eval":watch_list_eval}

f = open("results.txt","a") #opens file with name of "test.txt"
f.write(str(run_dict)+"\n"+"\n")
f.close()

#### Time  Taken

In [ ]:
print('Time Taken :'+str(datetime.datetime.now()-start_time))